In [ ]:
!ls /kaggle/input

In [ ]:
%%capture
!pip install efficientnet
!pip install tensorflow-addons
!pip install focal-loss
!pip install xlrd<=2.0
!pip install openpyxl 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf, tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense,Conv2D,Concatenate,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from kaggle_datasets import KaggleDatasets
import math
import tensorflow_addons as tfa


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

GCS_DS_PATH = KaggleDatasets().get_gcs_path('ocular-disease-recognition-odir5k')
print(GCS_DS_PATH)

In [ ]:
strategy.num_replicas_in_sync

In [ ]:
img_shape=456
BATCH_SIZE = 5*strategy.num_replicas_in_sync
BATCH_SIZE

In [ ]:
def train_format_path(st):
    return GCS_DS_PATH + '/ODIR-5K/ODIR-5K/Training Images/' + st 

def test_format_path(st):
    return GCS_DS_PATH + '/ODIR-5K/ODIR-5K/Testing Images/' + st 


In [ ]:
train = pd.read_excel('../input/odir-labels/ODIR-5K_Training_Annotations(Updated)_V2.xlsx')
train.head()

In [ ]:
test = pd.read_csv('../input/odir-labels/XYZ_ODIR.csv')
test.head()

In [ ]:
left_test=[]
right_test=[]
ID=[]
for i in test['ID']:
  ID.append(i)
  left_test.append(str(i)+'_left.jpg')
  right_test.append(str(i)+'_right.jpg')

name_df=pd.DataFrame(zip(ID,left_test,right_test),columns=['ID','Left-Fundus','Right-Fundus'])
test = pd.merge(name_df, test, on='ID')
test.head()

In [ ]:
test_labels=test[['N','D','G','C','A','H','M','O']]

In [ ]:
left_train=train['Left-Fundus']
right_train=train['Right-Fundus']

left_test=test['Left-Fundus']
right_test=test['Right-Fundus']

In [ ]:
len(left_train),len(right_train),len(left_test),len(right_test)

In [ ]:
left_train_paths = left_train.apply(train_format_path).values
right_train_paths = right_train.apply(train_format_path).values

left_test_paths = left_test.apply(test_format_path).values
right_test_paths = right_test.apply(test_format_path).values




In [ ]:
right_test_paths[0]

In [ ]:
train_labels = train[['N','D','G','C','A','H','M','O']]
train_labels.head()

In [ ]:
def decode(img,image_size=(img_shape, img_shape)):
    bits = tf.io.read_file(img)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    image = tf.image.random_flip_left_right(image, seed=2020)
    image = tf.image.random_flip_up_down(image, seed=2020)
    image = tf.image.random_crop(image,size=[img_shape,img_shape,3],seed=2020 )
    image = tf.image.rot90(image)
    return image

def decode_image(left, right,labels=None ):
    if labels is None:
        return tf.stack([decode(left),decode(right)])
    else:
        return tf.stack([decode(left),decode(right)]),labels 
    


In [ ]:
import efficientnet.tfkeras as efn
import tensorflow as tf, tensorflow.keras.backend as K
from tensorflow.keras.layers import concatenate,Activation,GlobalMaxPooling2D,Flatten,Dense,Dropout,Input,Reshape,Lambda,GlobalMaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras import optimizers
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,cohen_kappa_score,roc_auc_score


In [ ]:
def outer_product(x):
    #Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
    phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])
    
    # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
    phi_I = tf.reshape(phi_I,[-1,x[0].shape[3]*x[1].shape[3]])
    
    # Divide by feature map size [sizexsize]
    size1 = int(x[1].shape[1])
    size2 = int(x[1].shape[2])
    phi_I = tf.divide(phi_I, size1*size2)
    
    # Take signed square root of phi_I
    y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))
    
    # Apply l2 normalization
    z_l2 = tf.nn.l2_normalize(y_ssqrt, axis=1)
    return z_l2

In [ ]:
def parallel_model():
    input_layer = Input(shape=(2, img_shape,img_shape,3))
    left_input, right_input = Lambda(lambda x: tf.split(x, 2, axis=1))(input_layer)

    left_input = Reshape([img_shape, img_shape, 3])(left_input)
    right_input = Reshape([img_shape, img_shape, 3])(right_input)



    left_model = efn.EfficientNetB5(input_shape =  (img_shape,img_shape,3), include_top = False, weights = None,input_tensor=left_input)
    xl = left_model.output
    xl = GlobalMaxPooling2D()(xl)
    out1 = Dense(5, activation='softmax')(xl)
    left_model=tf.keras.Model(inputs=left_model.input, outputs=out1)
    left_model.load_weights('../input/aptos-model/aptosb5_model.hdf5')
    left_model=tf.keras.Model(left_model.input, left_model.layers[-5].output)
    
    
    
    right_model = efn.EfficientNetB5(input_shape =  (img_shape,img_shape,3), include_top = False, weights = None,input_tensor=right_input)
    xr = right_model.output
    xr = GlobalMaxPooling2D()(xr)
    outr = Dense(5, activation='softmax')(xr)
    right_model=tf.keras.Model(inputs=right_model.input, outputs=outr)
    right_model.load_weights('../input/aptos-model/aptosb5_model.hdf5')
    right_model=tf.keras.Model(right_model.input, right_model.layers[-5].output)

    for layer in right_model._layers:
        layer._name = layer._name + '_right'
    for layer in left_model._layers:
        layer._name = layer._name + '_left'

    left_model._name="left_eff"
    right_model._name="right_eff"


    left_model.compile(Adam(lr=0.001, decay=1e-3),loss='binary_crossentropy')
    right_model.compile(Adam(lr=0.001, decay=1e-3),loss='binary_crossentropy')
    
    d1=left_model.output
    d2=right_model.output
    
    bilinear = Lambda(outer_product, name='outer_product1')([d1,d2])
    predictions=Dense(8, activation='sigmoid', name='predictions')(bilinear)
    model = Model(inputs=right_model.input, outputs=predictions)

    return model


In [ ]:
from tensorflow.keras.optimizers import Adam
from focal_loss import BinaryFocalLoss

def get_model():
    opt = Adam(lr=0.0003, decay=1e-3)

    with strategy.scope():
        model = parallel_model()
        #model.summary()

    model.compile(optimizer=opt, loss=BinaryFocalLoss(gamma=5))
    return model

In [ ]:
# from sklearn.model_selection import train_test_split
# right_train,right_val, train_label_right, val_label_right= train_test_split(right_train_paths,train_labels, test_size = 500/3500,  random_state = 73)
# left_train, left_val,train_label_left, val_label_left = train_test_split(left_train_paths,train_labels, test_size = 500/3500,  random_state = 73)

In [ ]:

    
train_image=tf.data.Dataset.from_tensor_slices((left_train_paths,right_train_paths,train_labels ))
train_dataset=train_image.map(decode_image, num_parallel_calls=AUTO).repeat().shuffle(512).batch(BATCH_SIZE).prefetch(AUTO)

# val_image=tf.data.Dataset.from_tensor_slices((left_val,right_val,val_label_left ))
# val_dataset=train_image.map(decode_image, num_parallel_calls=AUTO).repeat().batch(BATCH_SIZE).prefetch(AUTO)

test_image=tf.data.Dataset.from_tensor_slices((left_test_paths,right_test_paths ))
test_dataset=test_image.map(decode_image, num_parallel_calls=AUTO).batch(BATCH_SIZE)


model=get_model()
history = model.fit(train_dataset,
                steps_per_epoch=train_labels.shape[0]/BATCH_SIZE,
                epochs=10,
                verbose=1,
                )
    



In [ ]:
y_pred=model.predict(test_dataset,steps=(len(test_labels)/BATCH_SIZE))
y_pred.shape,test_labels.shape

In [ ]:
for i,j in enumerate(['N','D','G','C','A','H','M','O']):
    test[j]=y_pred[:,i]

In [ ]:
test.drop(['Left-Fundus','Right-Fundus'],axis=1,inplace=True)

In [ ]:
test.to_csv('tpu_B5_pretrained_outer.csv',index=False)